In [12]:
from typing import List, Tuple, Optional
import math

# -----------------------------
# Spausdinimo pagalbinės
# -----------------------------
def print_matrix(M: List[List[float]], title: str = "", fmt: str = "{:11.6g}"):
    if title:
        print(f"\n{title}\n" + "-" * len(title))
    for row in M:
        print(" ".join(fmt.format(v) for v in row))
    print()

def print_vector(v: List[float], title: str = "", fmt: str = "{:11.6g}"):
    if title:
        print(f"\n{title}\n" + "-" * len(title))
    print(" ".join(fmt.format(x) for x in v))
    print()

def deep_copy_matrix(A: List[List[float]]) -> List[List[float]]:
    return [row[:] for row in A]

def augment(A: List[List[float]], b: List[float]) -> List[List[float]]:
    return [row + [b[i]] for i, row in enumerate(A)]

def mat_vec(A: List[List[float]], x: List[float]) -> List[float]:
    return [sum(A[i][j]*x[j] for j in range(len(x))) for i in range(len(A))]

def vec_sub(a: List[float], b: List[float]) -> List[float]:
    return [a[i]-b[i] for i in range(len(a))]

def norm2(v: List[float]) -> float:
    return math.sqrt(sum(x*x for x in v))

def check_solution(A: List[List[float]], x: Optional[List[float]], b: List[float], tag: str):
    if x is None:
        print(f"[{tag}] sprendinio nėra ką tikrinti (statusas ≠ 'unique').")
        return
    Ax = mat_vec(A, x)
    r  = vec_sub(b, Ax)
    print_vector(Ax, f"{tag}: A*x")
    print_vector(r,  f"{tag}: liekana r = b - A*x")
    print(f"{tag}: ||x||2 = {norm2(x):.12g},  ||r||2 = {norm2(r):.12g}\n")

# ============================================================
# 1) KRAMERIO METODAS (tik kvadratinėms, det(A)≠0)
# ============================================================
def det_gauss(A_in: List[List[float]], verbose: bool = False) -> float:
    """Determinantas per Gauso trianguliaciją su daliniu perstatymu."""
    A = deep_copy_matrix(A_in)
    n = len(A)
    det = 1.0
    swaps = 0
    for k in range(n):
        pivot = max(range(k, n), key=lambda i: abs(A[i][k]))
        if abs(A[pivot][k]) < 1e-15:
            return 0.0
        if pivot != k:
            A[k], A[pivot] = A[pivot], A[k]
            swaps ^= 1
            if verbose:
                print(f"[det] sukeista {k}↔{pivot}")
        det *= A[k][k]
        for i in range(k+1, n):
            m = A[i][k]/A[k][k]
            for j in range(k+1, n):
                A[i][j] -= m*A[k][j]
    return -det if swaps else det

def cramers_rule(A_in: List[List[float]], b: List[float], verbose: bool=True) -> Optional[List[float]]:
    n = len(A_in)
    A = deep_copy_matrix(A_in)
    if verbose:
        print_matrix(A, "Krameris: pradinė A")
        print_vector(b, "Krameris: pradinė b")

    D = det_gauss(A, verbose=False)
    print(f"Krameris: det(A) = {D:.12g}")
    if abs(D) < 1e-15:
        print("Krameris: det(A)=0 → metodas netaikomas (sprendinio nėra arba jų begalybė).")
        return None

    x = [0.0]*n
    for k in range(n):
        Ak = deep_copy_matrix(A)
        for i in range(n):
            Ak[i][k] = b[i]
        if verbose:
            print_matrix(Ak, f"Krameris: A^{k} (stulp. {k} pakeistas b)")
        Dk = det_gauss(Ak, verbose=False)
        x[k] = Dk / D
        if verbose:
            print(f"  x[{k}] = det(A_k)/det(A) = {Dk:.12g}/{D:.12g} = {x[k]:.12g}")
    print_vector(x, "Krameris: sprendinys x")
    check_solution(A, x, b, "Krameris")
    return x

# ============================================================
# 2) GAUSO METODAS su daliniu perstatymu.
#     Grąžina statusą: 'unique' / 'none' / 'infinite'
#     Jei 'unique' – grąžina x; jei 'infinite' – pateikia 1 bazinį sprendinį (laisvieji=0)
# ============================================================
def gauss_elimination(A_in: List[List[float]], b_in: List[float], verbose: bool=True):
    A = deep_copy_matrix(A_in)
    b = b_in[:]
    n = len(A)
    Ab = augment(A, b)
    if verbose:
        print_matrix(Ab, "Gausas: pradinė išplėstoji [A|b]")

    # Tiesioginė eiga su pivoting
    r = 0  # dabartinė eilutė
    for c in range(n):
        # surasti vedantįjį nuo r eilutės žemyn
        pivot = max(range(r, n), key=lambda i: abs(Ab[i][c]))
        if abs(Ab[pivot][c]) < 1e-15:
            continue
        if pivot != r:
            Ab[r], Ab[pivot] = Ab[pivot], Ab[r]
            if verbose:
                print(f"Gausas: sukeista {r}↔{pivot} stulp. {c}")
                print_matrix(Ab, "Po sukeitimo")
        # nulinam žemiau
        for i in range(r+1, n):
            m = Ab[i][c]/Ab[r][c]
            if abs(m) > 0:
                for j in range(c, n+1):
                    Ab[i][j] -= m*Ab[r][j]
                if verbose:
                    print(f"Gausas: i={i}, m={m:.6g} (nulinam stulp. {c})")
                    print_matrix(Ab, f"Po nulinimo i={i}")
        r += 1
        if r == n:
            break

    if verbose:
        print_matrix(Ab, "Gausas: viršutinė-trikampė (po trianguliacijos)")

    # Patikra: prieštaros / rangai
    rankA = 0
    rankAb = 0
    for i in range(n):
        rowA_zero = all(abs(Ab[i][j]) < 1e-12 for j in range(n))
        rowAb_zero = rowA_zero and abs(Ab[i][n]) < 1e-12
        if not rowA_zero:
            rankA += 1
        if not rowAb_zero:
            rankAb += 1

    if rankA < rankAb:
        print("Gausas: r(A) < r([A|b]) → sprendinių NĖRA.")
        return {'status': 'none', 'x': None, 'Ab': Ab}

    if rankA < n:
        print("Gausas: r(A) < n ir r(A)=r([A|b]) → BE GALO DAUG sprendinių.")
        # Pateiksime vieną bazinį sprendinį: tęsiame į RREF ir laisvuosius = 0
        x = rref_solve_basic_solution(Ab, verbose=verbose)
        check_solution(A_in, x, b_in, "Gausas (∞ sprendinių – bazinis)")
        return {'status': 'infinite', 'x': x, 'Ab': Ab}

    # Atgalinė eiga (unikalus atvejis)
    x = back_substitution_upper(Ab, verbose=verbose)
    print_vector(x, "Gausas: sprendinys x (unikalus)")
    check_solution(A_in, x, b_in, "Gausas")
    return {'status': 'unique', 'x': x, 'Ab': Ab}

def back_substitution_upper(Ab: List[List[float]], verbose: bool=True) -> List[float]:
    n = len(Ab)
    x = [0.0]*n
    if verbose:
        print_matrix(Ab, "Atgalinė eiga: naudojam [U|y]")
    for i in range(n-1, -1, -1):
        s = sum(Ab[i][j]*x[j] for j in range(i+1, n))
        if abs(Ab[i][i]) < 1e-15:
            raise ZeroDivisionError("Atgalinė eiga: Įstrižainė ≈ 0 (unikalumui būtina).")
        x[i] = (Ab[i][n] - s)/Ab[i][i]
        if verbose:
            print(f"  x[{i}] = (b~[{i}] - suma)/U[{i},{i}] = {(Ab[i][n]-s):.6g}/{Ab[i][i]:.6g} = {x[i]:.12g}")
    return x

def rref_solve_basic_solution(Ab_in: List[List[float]], verbose: bool=True) -> List[float]:
    """RREF su laisvais = 0 → bazinis sprendinys, jei sistema suderinama."""
    Ab = deep_copy_matrix(Ab_in)
    n = len(Ab)
    m = len(Ab[0]) - 1
    r = c = 0
    if verbose:
        print_matrix(Ab, "RREF (startas)")
    while r < n and c < m:
        pivot = max(range(r, n), key=lambda i: abs(Ab[i][c]))
        if abs(Ab[pivot][c]) < 1e-15:
            c += 1
            continue
        if pivot != r:
            Ab[r], Ab[pivot] = Ab[pivot], Ab[r]
            if verbose:
                print(f"RREF: sukeista {r}↔{pivot} (c={c})")
                print_matrix(Ab, "Po sukeitimo")
        fac = Ab[r][c]
        for j in range(c, m+1):
            Ab[r][j] /= fac
        if verbose:
            print_matrix(Ab, f"RREF: normalizuota r={r}")
        for i in range(n):
            if i == r: 
                continue
            mlt = Ab[i][c]
            for j in range(c, m+1):
                Ab[i][j] -= mlt*Ab[r][j]
        if verbose:
            print_matrix(Ab, f"RREF: nulinimai stulpelyje c={c}")
        r += 1; c += 1
    # bazinis sprendinys: x = (laisvus=0)
    x = [0.0]*m
    pivot_cols = []
    row_pivot_col = {}
    for i in range(n):
        pc = next((j for j in range(m) if abs(Ab[i][j]-1) < 1e-12 and all(abs(Ab[i][jj])<1e-12 for jj in range(j+1, m))), None)
        # paprastesnė paieška: tikrinam kas yra 1 ir vienintelė ne-nulinė
        if pc is None:
            # robust: surask 1 ir stulpelyje kitur 0
            for j in range(m):
                if abs(Ab[i][j]-1) < 1e-12 and all(abs(Ab[ii][j])<1e-12 for ii in range(n) if ii!=i):
                    pc = j; break
        if pc is not None:
            pivot_cols.append(pc)
            row_pivot_col[i] = pc
    for i, pc in row_pivot_col.items():
        x[pc] = Ab[i][m]
    print_vector(x, "RREF: vienas bazinis sprendinys (laisvieji=0)")
    return x

# ============================================================
# 3) GAUSO–ŽORDANO METODAS (RREF): tiesiogiai sprendžia Ax=b
# ============================================================
def gauss_jordan(A_in: List[List[float]], b_in: List[float], verbose: bool=True) -> dict:
    A = deep_copy_matrix(A_in)
    b = b_in[:]
    n = len(A)
    Ab = augment(A, b)
    if verbose:
        print_matrix(Ab, "G–Ž: pradinė [A|b]")

    r = c = 0
    while r < n and c < n:
        pivot = max(range(r, n), key=lambda i: abs(Ab[i][c]))
        if abs(Ab[pivot][c]) < 1e-15:
            c += 1
            continue
        if pivot != r:
            Ab[r], Ab[pivot] = Ab[pivot], Ab[r]
            if verbose:
                print(f"G–Ž: sukeista {r}↔{pivot}")
                print_matrix(Ab, "Po sukeitimo")

        fac = Ab[r][c]
        for j in range(c, n+1):
            Ab[r][j] /= fac
        if verbose:
            print_matrix(Ab, f"G–Ž: normalizuota r={r}")

        for i in range(n):
            if i == r: 
                continue
            mlt = Ab[i][c]
            for j in range(c, n+1):
                Ab[i][j] -= mlt*Ab[r][j]
        if verbose:
            print_matrix(Ab, f"G–Ž: nulinimai c={c}")

        r += 1; c += 1

    # statusas
    rankA = sum(any(abs(Ab[i][j])>1e-12 for j in range(n)) for i in range(n))
    inconsistent = any(all(abs(Ab[i][j])<1e-12 for j in range(n)) and abs(Ab[i][n])>1e-12 for i in range(n))
    if inconsistent:
        print("G–Ž: sprendinių NĖRA.")
        return {'status': 'none', 'x': None, 'Ab': Ab}
    if rankA < n:
        print("G–Ž: BE GALO DAUG sprendinių (laisvieji kintamieji).")
        x = rref_solve_basic_solution(Ab, verbose=verbose)
        check_solution(A_in, x, b_in, "G–Ž (∞ sprendinių – bazinis)")
        return {'status': 'infinite', 'x': x, 'Ab': Ab}

    x = [Ab[i][n] for i in range(n)]
    print_vector(x, "G–Ž: sprendinys x (unikalus)")
    check_solution(A_in, x, b_in, "G–Ž")
    return {'status': 'unique', 'x': x, 'Ab': Ab}

# ============================================================
# 4) ATVIRKŠTINĖS MATRICOS METODAS: x = A^{-1} b
#    (A^{-1} skaičiuojame per [A|I] → RREF)
# ============================================================
def inverse_via_gauss_jordan(A_in: List[List[float]], verbose: bool=True) -> Optional[List[List[float]]]:
    A = deep_copy_matrix(A_in)
    n = len(A)
    # [A | I]
    AI = [A[i] + [1.0 if i==j else 0.0 for j in range(n)] for i in range(n)]
    if verbose:
        print_matrix(AI, "Atvirkštinė: pradinė [A|I]")

    r = c = 0
    while r < n and c < n:
        pivot = max(range(r, n), key=lambda i: abs(AI[i][c]))
        if abs(AI[pivot][c]) < 1e-15:
            print("Atvirkštinė: A neatgręžiama (det=0).")
            return None
        if pivot != r:
            AI[r], AI[pivot] = AI[pivot], AI[r]
            if verbose:
                print(f"[Inv] sukeista {r}↔{pivot}")
                print_matrix(AI, "Po sukeitimo")

        fac = AI[r][c]
        for j in range(c, 2*n):
            AI[r][j] /= fac
        if verbose:
            print_matrix(AI, f"[Inv] normalizuota r={r}")

        for i in range(n):
            if i == r: 
                continue
            mlt = AI[i][c]
            for j in range(c, 2*n):
                AI[i][j] -= mlt*AI[r][j]
        if verbose:
            print_matrix(AI, f"[Inv] nulinimai c={c}")
        r += 1; c += 1

    Ainv = [row[n:] for row in AI]
    print_matrix(Ainv, "Gauta A^{-1}")
    return Ainv

def inverse_matrix_method(A_in: List[List[float]], b: List[float], verbose: bool=True) -> Optional[List[float]]:
    if verbose:
        print_matrix(A_in, "Atvirkštinės metodas: pradinė A")
        print_vector(b, "Atvirkštinės metodas: pradinė b")
    Ainv = inverse_via_gauss_jordan(A_in, verbose=verbose)
    if Ainv is None:
        print("Atvirkštinės metodas: sprendinys neegzistuoja (A neatgręžiama).")
        return None
    # x = A^{-1} b
    x = [sum(Ainv[i][j]*b[j] for j in range(len(b))) for i in range(len(Ainv))]
    print_vector(x, "Atvirkštinės metodas: sprendinys x")
    check_solution(A_in, x, b, "Atvirkštinės metodas")
    return x

# ============================================================
# 5) LU SKAIDA (PA=LU) + sprendimas
# ============================================================
def lu_decomposition(A_in: List[List[float]], verbose: bool=True) -> Tuple[List[List[float]], List[List[float]], List[List[float]]]:
    A = deep_copy_matrix(A_in)
    n = len(A)
    U = deep_copy_matrix(A)
    L = [[0.0]*n for _ in range(n)]
    P = [[1.0 if i==j else 0.0 for j in range(n)] for i in range(n)]
    for k in range(n):
        L[k][k] = 1.0

    if verbose:
        print_matrix(A, "LU: pradinė A")

    for k in range(n-1):
        pivot = max(range(k, n), key=lambda i: abs(U[i][k]))
        if abs(U[pivot][k]) < 1e-15:
            raise ZeroDivisionError("LU: singuliari matrica (vedantysis≈0).")
        if pivot != k:
            U[k], U[pivot] = U[pivot], U[k]
            P[k], P[pivot] = P[pivot], P[k]
            for j in range(k):
                L[k][j], L[pivot][j] = L[pivot][j], L[k][j]
            if verbose:
                print(f"LU: sukeista {k}↔{pivot}")
                print_matrix(U, "LU: U po sukeitimo")
                print_matrix(L, "LU: L po sukeitimo")
                print_matrix(P, "LU: P po sukeitimo")

        for i in range(k+1, n):
            L[i][k] = U[i][k]/U[k][k]
            U[i][k] = 0.0
            for j in range(k+1, n):
                U[i][j] -= L[i][k]*U[k][j]
            if verbose:
                print(f"LU: i={i}, L[{i},{k}]={L[i][k]:.6g}")
                print_matrix(U, f"LU: U po nulinimo i={i}, k={k}")
                print_matrix(L, f"LU: L po atnaujinimo i={i}, k={k}")
    if verbose:
        print_matrix(L, "LU: galutinė L")
        print_matrix(U, "LU: galutinė U")
        print_matrix(P, "LU: galutinė P")
    return P, L, U

def forward_substitution(L: List[List[float]], b: List[float], verbose: bool=True) -> List[float]:
    n = len(L)
    y = [0.0]*n
    if verbose:
        print_matrix(L, "Priekinė eiga: L")
        print_vector(b, "Priekinė eiga: b")
    for i in range(n):
        s = sum(L[i][j]*y[j] for j in range(i))
        if abs(L[i][i]) < 1e-15:
            raise ZeroDivisionError("Priekinė eiga: L[i,i]=0")
        y[i] = (b[i]-s)/L[i][i]
        if verbose:
            print(f"  y[{i}] = {(b[i]-s):.6g}/{L[i][i]:.6g} = {y[i]:.12g}")
    print_vector(y, "Priekinė eiga: y")
    return y

def back_substitution(U: List[List[float]], y: List[float], verbose: bool=True) -> List[float]:
    n = len(U)
    x = [0.0]*n
    if verbose:
        print_matrix(U, "Atgalinė eiga: U")
        print_vector(y, "Atgalinė eiga: y")
    for i in range(n-1, -1, -1):
        s = sum(U[i][j]*x[j] for j in range(i+1, n))
        if abs(U[i][i]) < 1e-15:
            raise ZeroDivisionError("Atgalinė eiga: U[i,i]=0")
        x[i] = (y[i]-s)/U[i][i]
        if verbose:
            print(f"  x[{i}] = {(y[i]-s):.6g}/{U[i][i]:.6g} = {x[i]:.12g}")
    print_vector(x, "Atgalinė eiga: x")
    return x

def lu_solve(P: List[List[float]], L: List[List[float]], U: List[List[float]], b: List[float], verbose: bool=True) -> List[float]:
    # P*b
    Pb = [sum(P[i][j]*b[j] for j in range(len(b))) for i in range(len(b))]
    if verbose:
        print_vector(b, "LU sprendimas: pradinė b")
        print_vector(Pb, "LU sprendimas: P*b")
    y = forward_substitution(L, Pb, verbose=verbose)
    x = back_substitution(U, y, verbose=verbose)
    print_vector(x, "LU sprendimas: sprendinys x")
    return x

def lu_method(A_in: List[List[float]], b: List[float], verbose: bool=True) -> Optional[List[float]]:
    try:
        P, L, U = lu_decomposition(A_in, verbose=verbose)
        x = lu_solve(P, L, U, b, verbose=verbose)
        check_solution(A_in, x, b, "LU")
        return x
    except ZeroDivisionError as e:
        print(f"LU: klaida – {e}")
        return None

# ============================================================
# MAIN – čia tik UŽKOMENTUOTI paleidimai (atsikomentuok ko reikia)
# ============================================================
def main():
    # PAGRINDINIS PAVYZDYS
    A = [
        [1.0,  1.0, 1.0],
        [1.0,  -1.0, -1],
        [2.0, 1.0, -1.0],
    ]
    b = [1.0 , 1.0, 1.0]

    # --- KRAMERIS ---
    # x_cramer = cramers_rule(A, b, verbose=True)

    # --- GAUSAS (unikalus / nėra / begaliniai) ---
    # rez_gauss = gauss_elimination(A, b, verbose=True)
    # x_gauss = rez_gauss['x']

    # --- GAUSO–ŽORDANO ---
    # rez_gj = gauss_jordan(A, b, verbose=True)
    # x_gj = rez_gj['x']

    # --- ATVIRKŠTINĖS MATRICOS METODAS ---
    # x_inv = inverse_matrix_method(A, b, verbose=True)

    # --- LU SKAIDA ---
    x_lu = lu_method(A, b, verbose=True)

    pass

main()


LU: pradinė A
-------------
          1           1           1
          1          -1          -1
          2           1          -1

LU: sukeista 0↔2

LU: U po sukeitimo
------------------
          2           1          -1
          1          -1          -1
          1           1           1


LU: L po sukeitimo
------------------
          1           0           0
          0           1           0
          0           0           1


LU: P po sukeitimo
------------------
          0           0           1
          0           1           0
          1           0           0

LU: i=1, L[1,0]=0.5

LU: U po nulinimo i=1, k=0
--------------------------
          2           1          -1
          0        -1.5        -0.5
          1           1           1


LU: L po atnaujinimo i=1, k=0
-----------------------------
          1           0           0
        0.5           1           0
          0           0           1

LU: i=2, L[2,0]=0.5

LU: U po nulinimo i=2, k=0